In [5]:
import pandas as pd 
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,19.800000
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,6.600000
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,17.916667
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,8.300000
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,6.100000


In [9]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
train_dicts = df[categorical + numerical].to_dict(orient = 'records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)


target = 'duration'
Y_train = df[target].values
Y_train

lr = LinearRegression()
lr.fit(X_train,Y_train)

Y_pred = lr.predict(X_train)

mean_squared_error(Y_train, Y_pred)

63.23475630560298

import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(Y_pred, color='blue', label='Prediction', kde=True)
sns.histplot(Y_train, color='orange', label='Real Values', kde=True)

plt.legend()
plt.title("Distribution of Predicted vs Real Values")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()


In [10]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [11]:
n = len(df)

In [12]:
df['PU_DO'] = df['PULocationID']+ '_' + df['DOLocationID'] 

In [13]:

train_data = df.loc[:int(n*0.8)]
val_data = df.loc[int(n*0.8):]

len(train_data), len(val_data)

(2264655, 634252)

In [14]:
len(val_data.columns),len(train_data.columns)

(21, 21)

In [15]:
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = train_data[categorical + numerical ].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = val_data[categorical + numerical ].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

target = 'duration'
Y_train = train_data[target].values
Y_val = val_data[target].values 


lr = LinearRegression()
lr.fit(X_train,Y_train)


Y_pred = lr.predict(X_val)

mean_squared_error(Y_pred, Y_val)

58.34150056925522

In [16]:
# log model to mlflow 
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)



In [17]:
from sklearn.linear_model import Lasso

mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("nyc-taxi-experiment")

with mlflow.start_run():
    mlflow.set_tag("developer","bechir")

    mlflow.log_param("train-data-URL","https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
    alpha = 0.1 
    mlflow.log_param("alpha",alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, Y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(Y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    

🏃 View run intelligent-asp-503 at: http://127.0.0.1:5000/#/experiments/1/runs/0f32aa73c7374ea08747095b91842790
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [18]:
with mlflow.start_run():
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="simple LR model")

🏃 View run classy-sow-222 at: http://127.0.0.1:5000/#/experiments/1/runs/97cb21005c4745ffaba5200bf43aa001
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [19]:
import xgboost as xgb
from hyperopt import fmin , tpe , hp , Trials, STATUS_OK
from hyperopt.pyll import scope 


In [20]:
train = xgb.DMatrix(X_train, label=Y_train)
valid = xgb.DMatrix(X_val, label=Y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/bechir/.local/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [11:54:38] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.41135                           
[1]	validation-rmse:7.44828                           
[2]	validation-rmse:6.77903                           
[3]	validation-rmse:6.32559                           
[4]	validation-rmse:6.02545                           
[5]	validation-rmse:5.82964                           
[6]	validation-rmse:5.70348                           
[7]	validation-rmse:5.62254                           
[8]	validation-rmse:5.57114                           
[9]	validation-rmse:5.54085                           
[10]	validation-rmse:5.51748                          
[11]	validation-rmse:5.50441                          
[12]	validation-rmse:5.49556                          
[13]	validation-rmse:5.48921                          
[14]	validation-rmse:5.48405                          
[15]	validation-rmse:5.48033                          
[16]	validation-rmse:5.47729                          
[17]	validation-rmse:5.47443                          
[18]	valid

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

with open('models/preprocessor.b', 'wb') as f_out:
    pickle.dump(dv, f_out)

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, Y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(Y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2025/07/03 12:06:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: Unable to allocate 345. GiB for an array with shape (2264655, 20467) and data type float64
